In [1]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
import os.path as osp
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F



from config.config_factory import _C as cfg
from config.config_factory import build_output

from tools.logger import setup_logger
logger = setup_logger(".")
from tools.utils import deploy_macro

import cv2
from PIL import Image 

def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

from tools.utils import multi_pose_decode, multi_pose_post_process, _sigmoid

In [2]:
from database.dataset_factory import get_dataset
from manager.manager_factory import get_manager
from database.transform_factory import get_transform

In [3]:
from database.data_factory import get_names as data_names
from database.dataset_factory import get_names as dataset_names
from database.loader_factory import get_names as loader_names
from manager.manager_factory import get_names as manager_names
from engine.engine_factory import get_names as engine_names
print("DATA: ", data_names())
print("DATASET: ", dataset_names())
print("LOADER: ", loader_names())
print("MANAGER: ", manager_names())
print("ENGINE: ", engine_names())

DATA:  ['coco', 'coco_person_kp', 'deepfashion', 'market', 'cuhk01', 'cuhk02', 'cuhk03', 'duke', 'msmt', 'imagenet', 'cifar10']
DATASET:  ['coco_keypoint', 'coco_object', 'reid', 'imagenet', 'cifar10']
LOADER:  ['coco', 'reid', 'imagenet', 'cifar10']
MANAGER:  ['center', 'center_kp', 'imagenet', 'par', 'reid_trick']
ENGINE:  ['center', 'center_kp', 'imagenet', 'par', 'reid_trick']


In [3]:
config_path = '/media/allen/mass/deep-learning-works/config/cifar10.yml'
cfg.merge_from_file(config_path)
deploy_macro(cfg)

2020-02-04 17:20:04,555 logger INFO: Using GPU: 1


In [13]:
from database.transform_factory import get_transform
trans = get_transform(cfg, cfg.TRAIN_TRANSFORM)

In [6]:
trans.t_list

[AugMix, Tensorize]

In [15]:
from database.loader_factory import get_loader
loader = get_loader('cifar10')(cfg)

Files already downloaded and verified
2020-02-03 15:19:04,439 logger INFO: => CIFAR10 TRAIN loaded
2020-02-03 15:19:04,440 logger INFO: Dataset statistics:
2020-02-03 15:19:04,440 logger INFO:   ------------------------------
2020-02-03 15:19:04,441 logger INFO:   subset   | # class | # images
2020-02-03 15:19:04,441 logger INFO:   ------------------------------
2020-02-03 15:19:04,442 logger INFO:   train    |      10 |    50000
2020-02-03 15:19:04,442 logger INFO:   ------------------------------
Files already downloaded and verified
2020-02-03 15:19:05,103 logger INFO: => CIFAR10 VAL loaded
2020-02-03 15:19:05,103 logger INFO: Dataset statistics:
2020-02-03 15:19:05,104 logger INFO:   ------------------------------
2020-02-03 15:19:05,104 logger INFO:   subset   | # class | # images
2020-02-03 15:19:05,106 logger INFO:   ------------------------------
2020-02-03 15:19:05,107 logger INFO:   val      |      10 |    10000
2020-02-03 15:19:05,107 logger INFO:   -------------------------

In [16]:
batch = next(iter(loader['train']))

In [17]:
batch['inp'][0]

tensor([[[ 0.8000,  0.8000,  2.0591,  ...,  0.8000, -0.9285, -0.9285],
         [ 0.7932,  0.7891,  2.0482,  ...,  0.7891, -0.9326, -0.9326],
         [ 0.8000,  0.8000,  2.0591,  ...,  2.0591,  0.3306,  0.3306],
         ...,
         [ 1.9937,  1.4153,  0.6493,  ...,  2.0050, -1.2781, -1.2837],
         [ 0.7430,  0.2459, -0.3034,  ...,  2.0106, -1.2781, -1.2837],
         [ 0.7593,  0.4387,  0.1097,  ...,  2.0218, -1.2781, -1.2837]],

        [[ 0.8493,  0.8493,  2.1265,  ...,  0.8493, -0.9040, -0.9040],
         [ 0.8424,  0.8382,  2.1154,  ...,  0.8382, -0.9082, -0.9082],
         [ 0.8493,  0.8493,  2.1265,  ...,  2.1265,  0.3732,  0.3732],
         ...,
         [ 2.0601,  1.4634,  0.6530,  ...,  2.0793, -1.2587, -1.2615],
         [ 0.7914,  0.2789, -0.3278,  ...,  2.0821, -1.2615, -1.2615],
         [ 0.8080,  0.4662,  0.0912,  ...,  2.1017, -1.2615, -1.2587]],

        [[ 0.9270,  0.9270,  2.1158,  ...,  0.9270, -0.7050, -0.7050],
         [ 0.9206,  0.9167,  2.1055,  ...,  0

In [10]:
from database.data_factory import get_data
data = get_data('cifar10')(cfg)

Files already downloaded and verified
2020-02-03 14:28:47,055 logger INFO: => CIFAR10 TRAIN loaded
2020-02-03 14:28:47,056 logger INFO: Dataset statistics:
2020-02-03 14:28:47,057 logger INFO:   ------------------------------
2020-02-03 14:28:47,057 logger INFO:   subset   | # class | # images
2020-02-03 14:28:47,058 logger INFO:   ------------------------------
2020-02-03 14:28:47,059 logger INFO:   train    |      10 |    50000
2020-02-03 14:28:47,059 logger INFO:   ------------------------------
Files already downloaded and verified
2020-02-03 14:28:47,714 logger INFO: => CIFAR10 VAL loaded
2020-02-03 14:28:47,714 logger INFO: Dataset statistics:
2020-02-03 14:28:47,715 logger INFO:   ------------------------------
2020-02-03 14:28:47,716 logger INFO:   subset   | # class | # images
2020-02-03 14:28:47,716 logger INFO:   ------------------------------
2020-02-03 14:28:47,717 logger INFO:   val      |      10 |    10000
2020-02-03 14:28:47,717 logger INFO:   -------------------------

In [4]:
from trainer.trainer_factory import get_trainer
trainer = get_trainer(cfg.TRAINER)(cfg)

Files already downloaded and verified
2020-02-04 17:20:18,076 logger INFO: => CIFAR10 TRAIN loaded
2020-02-04 17:20:18,077 logger INFO: Dataset statistics:
2020-02-04 17:20:18,078 logger INFO:   ------------------------------
2020-02-04 17:20:18,078 logger INFO:   subset   | # class | # images
2020-02-04 17:20:18,079 logger INFO:   ------------------------------
2020-02-04 17:20:18,080 logger INFO:   train    |      10 |    50000
2020-02-04 17:20:18,081 logger INFO:   ------------------------------
Files already downloaded and verified
2020-02-04 17:20:18,783 logger INFO: => CIFAR10 VAL loaded
2020-02-04 17:20:18,783 logger INFO: Dataset statistics:
2020-02-04 17:20:18,784 logger INFO:   ------------------------------
2020-02-04 17:20:18,785 logger INFO:   subset   | # class | # images
2020-02-04 17:20:18,785 logger INFO:   ------------------------------
2020-02-04 17:20:18,787 logger INFO:   val      |      10 |    10000
2020-02-04 17:20:18,787 logger INFO:   -------------------------

In [7]:
trainer.manager.check_size((1,3,32,32))

2020-02-04 14:45:25,140 logger INFO: backbone.conv1.conv                                         
2020-02-04 14:45:25,141 logger INFO:                input,   1   3  32  32 
2020-02-04 14:45:25,141 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,143 logger INFO: backbone.conv1.bn                                           
2020-02-04 14:45:25,143 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,144 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,144 logger INFO: backbone.conv1.relu                                         
2020-02-04 14:45:25,145 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,146 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,147 logger INFO: backbone.conv1                                              
2020-02-04 14:45:25,148 logger INFO:                input,   1   3  32  32 
2020-02-04 14:45:25,149 logger INFO:               output,   1  64  32  32 


2020-02-04 14:45:25,226 logger INFO: backbone.conv2.0.conv2c.1.relu                              
2020-02-04 14:45:25,226 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,227 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,228 logger INFO: backbone.conv2.0.conv2c.1                                   
2020-02-04 14:45:25,229 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,229 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,230 logger INFO: backbone.conv2.0.conv2c.2.conv1                             
2020-02-04 14:45:25,230 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,231 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,231 logger INFO: backbone.conv2.0.conv2c.2.conv2                             
2020-02-04 14:45:25,232 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,233 logger INFO:               output,   1  64  32  32 


2020-02-04 14:45:25,302 logger INFO: backbone.conv2.0.gate.gate_activation                       
2020-02-04 14:45:25,302 logger INFO:                input,   1  64   1   1 
2020-02-04 14:45:25,303 logger INFO:               output,   1  64   1   1 
2020-02-04 14:45:25,305 logger INFO: backbone.conv2.0.gate                                       
2020-02-04 14:45:25,305 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,306 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,308 logger INFO: backbone.conv2.0.gate.global_avgpool                        
2020-02-04 14:45:25,308 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,309 logger INFO:               output,   1  64   1   1 
2020-02-04 14:45:25,310 logger INFO: backbone.conv2.0.gate.fc1                                   
2020-02-04 14:45:25,311 logger INFO:                input,   1  64   1   1 
2020-02-04 14:45:25,311 logger INFO:               output,   1   4   1   1 


2020-02-04 14:45:25,390 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,391 logger INFO: backbone.conv2.1.conv2a.bn                                  
2020-02-04 14:45:25,392 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,392 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,393 logger INFO: backbone.conv2.1.conv2a.relu                                
2020-02-04 14:45:25,394 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,395 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,395 logger INFO: backbone.conv2.1.conv2a                                     
2020-02-04 14:45:25,395 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,396 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,397 logger INFO: backbone.conv2.1.conv2b.0.conv1                             
2020-02-04 14:45:25,398 logger INFO:                input,   1  64  32  32 


2020-02-04 14:45:25,472 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,473 logger INFO: backbone.conv2.1.conv2d.0.relu                              
2020-02-04 14:45:25,474 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,475 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,476 logger INFO: backbone.conv2.1.conv2d.0                                   
2020-02-04 14:45:25,478 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,479 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,482 logger INFO: backbone.conv2.1.conv2d.1.conv1                             
2020-02-04 14:45:25,483 logger INFO:                input,   1  64  32  32 
2020-02-04 14:45:25,483 logger INFO:               output,   1  64  32  32 
2020-02-04 14:45:25,485 logger INFO: backbone.conv2.1.conv2d.1.conv2                             
2020-02-04 14:45:25,486 logger INFO:                input,   1  64  32  32 


...

2020-02-04 14:45:26,184 logger INFO: backbone.conv4.0.conv2c.2.conv1                             
2020-02-04 14:45:26,184 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,185 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,186 logger INFO: backbone.conv4.0.conv2c.2.conv2                             
2020-02-04 14:45:26,186 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,187 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,189 logger INFO: backbone.conv4.0.conv2c.2.bn                                
2020-02-04 14:45:26,190 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,191 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,192 logger INFO: backbone.conv4.0.conv2c.2.relu                              
2020-02-04 14:45:26,192 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,193 logger INFO:               output,   1 128   8   8 


2020-02-04 14:45:26,277 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,278 logger INFO: backbone.conv4.0.gate.global_avgpool                        
2020-02-04 14:45:26,279 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,280 logger INFO:               output,   1 128   1   1 
2020-02-04 14:45:26,282 logger INFO: backbone.conv4.0.gate.fc1                                   
2020-02-04 14:45:26,283 logger INFO:                input,   1 128   1   1 
2020-02-04 14:45:26,284 logger INFO:               output,   1   8   1   1 
2020-02-04 14:45:26,286 logger INFO: backbone.conv4.0.gate.relu                                  
2020-02-04 14:45:26,287 logger INFO:                input,   1   8   1   1 
2020-02-04 14:45:26,287 logger INFO:               output,   1   8   1   1 
2020-02-04 14:45:26,289 logger INFO: backbone.conv4.0.gate.fc2                                   
2020-02-04 14:45:26,289 logger INFO:                input,   1   8   1   1 


2020-02-04 14:45:26,361 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,362 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,363 logger INFO: backbone.conv4.1.conv2a                                     
2020-02-04 14:45:26,363 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,364 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,366 logger INFO: backbone.conv4.1.conv2b.0.conv1                             
2020-02-04 14:45:26,367 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,368 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,369 logger INFO: backbone.conv4.1.conv2b.0.conv2                             
2020-02-04 14:45:26,369 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,370 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,371 logger INFO: backbone.conv4.1.conv2b.0.bn                                


2020-02-04 14:45:26,443 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,444 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,445 logger INFO: backbone.conv4.1.conv2d.1.conv1                             
2020-02-04 14:45:26,446 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,446 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,448 logger INFO: backbone.conv4.1.conv2d.1.conv2                             
2020-02-04 14:45:26,448 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,449 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,450 logger INFO: backbone.conv4.1.conv2d.1.bn                                
2020-02-04 14:45:26,451 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,451 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,452 logger INFO: backbone.conv4.1.conv2d.1.relu                              


2020-02-04 14:45:26,518 logger INFO:                input,   1 128   1   1 
2020-02-04 14:45:26,518 logger INFO:               output,   1 128   1   1 
2020-02-04 14:45:26,520 logger INFO: backbone.conv4.1.gate                                       
2020-02-04 14:45:26,520 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,521 logger INFO:               output,   1 128   8   8 
2020-02-04 14:45:26,522 logger INFO: backbone.conv4.1.gate.global_avgpool                        
2020-02-04 14:45:26,522 logger INFO:                input,   1 128   8   8 
2020-02-04 14:45:26,523 logger INFO:               output,   1 128   1   1 
2020-02-04 14:45:26,524 logger INFO: backbone.conv4.1.gate.fc1                                   
2020-02-04 14:45:26,525 logger INFO:                input,   1 128   1   1 
2020-02-04 14:45:26,526 logger INFO:               output,   1   8   1   1 
2020-02-04 14:45:26,527 logger INFO: backbone.conv4.1.gate.relu                                  


In [73]:
class LinkedNode:
    def __init__(self, name, outsize):
        self.name = name
        self.outsize = outsize
        self.next = []
    def link(self, node):
        self.next.append(node)
    
    def __repr__(self):
        return self.name
inps = {}
outs = {}
def check_size_hooker(m, inp, out):
    print(f'{m.name}----')

    is_link = False
    if isinstance(inp, tuple):            
        for i in inp:
            attr = getattr(i, 'node', None)
            if attr is not None:                    
                out.node = LinkedNode(m.name, out.shape)
                i.node.link(out.node)
                is_link = True
                print('    link')
    else:
        attr = getattr(inp, 'node', None)
        if attr is not None:                    
            out.node = LinkedNode(m.name, out.shape)
            inp.node.link(out.node)
            is_link = True
            print('    link')
    if not is_link and isinstance(out, torch.Tensor):
        out.node = LinkedNode(m.name, out.shape)
        print('    build')
    print('')
    inps[m.name] = inp
    outs[m.name] = out

In [74]:
from copy import deepcopy
dummy_model = deepcopy(trainer.manager.model) 

In [75]:


hooks = []
for n, m in dummy_model.named_modules():
    m.name = n
    handle = m.register_forward_hook(check_size_hooker)
    hooks.append(handle)
      
weight = next(iter(dummy_model.parameters()))


In [76]:
insize = (1,3,32,32)
if weight.is_cuda:
    dummy_input = torch.rand(insize).cuda()
else:
    dummy_input = torch.rand(insize)
dummy_input.node = LinkedNode('input', dummy_input)
dummy_model.eval()
with torch.no_grad():
    dummy_model(dummy_input)

for hook in hooks:
    hook.remove()

backbone.conv1----
    link

backbone.bn1----
    link

backbone.layer1.0.conv1----
    build

backbone.layer1.0.bn1----
    link

backbone.layer1.0.conv2----
    build

backbone.layer1.0.bn2----
    link

backbone.layer1.0.shortcut----
    build

backbone.layer1.0----
    link

backbone.layer1.1.conv1----
    link

backbone.layer1.1.bn1----
    link

backbone.layer1.1.conv2----
    build

backbone.layer1.1.bn2----
    link

backbone.layer1.1.shortcut----
    link

backbone.layer1.1----
    link

backbone.layer1----
    link

backbone.layer2.0.conv1----
    link

backbone.layer2.0.bn1----
    link

backbone.layer2.0.conv2----
    build

backbone.layer2.0.bn2----
    link

backbone.layer2.0.shortcut.0----
    link

backbone.layer2.0.shortcut.1----
    link

backbone.layer2.0.shortcut----
    link

backbone.layer2.0----
    link

backbone.layer2.1.conv1----
    link

backbone.layer2.1.bn1----
    link

backbone.layer2.1.conv2----
    build

backbone.layer2.1.bn2----
    link

backbone.la

In [84]:
torch.abs(outs['backbone.bn1'] - inps['backbone.layer1.0.conv1'][0]).sum()

tensor(5017.4658, device='cuda:0')

tensor([[[[2.4088e-01, 1.9228e-01, 1.5411e-01,  ..., 2.1228e-01,
           1.1448e-01, 7.0002e-02],
          [3.0273e-01, 2.5926e-01, 2.3958e-01,  ..., 6.0269e-02,
           1.2189e-01, 1.2415e-01],
          [3.2554e-01, 2.6568e-01, 1.3921e-01,  ..., 2.8870e-01,
           6.8208e-02, 0.0000e+00],
          ...,
          [2.7118e-01, 1.5042e-01, 3.2408e-01,  ..., 1.1693e-01,
           2.9643e-01, 2.2218e-01],
          [1.9630e-01, 4.5053e-02, 2.2368e-01,  ..., 1.5677e-01,
           1.6709e-01, 1.7482e-01],
          [1.1897e-01, 2.2359e-01, 1.5601e-01,  ..., 1.1885e-01,
           2.7661e-01, 6.4512e-03]],

         [[3.6215e-02, 3.5311e-02, 2.8695e-02,  ..., 2.5804e-02,
           7.0063e-02, 0.0000e+00],
          [9.2578e-02, 8.2682e-02, 4.2349e-02,  ..., 5.6060e-02,
           0.0000e+00, 0.0000e+00],
          [1.1066e-02, 5.5565e-02, 0.0000e+00,  ..., 2.5432e-02,
           3.9870e-02, 0.0000e+00],
          ...,
          [2.0745e-02, 0.0000e+00, 6.9587e-02,  ..., 0.0000

In [72]:
start_node = dummy_input.node
while len(start_node.next) > 0:
    print(start_node.next)
    start_node = start_node.next[0]
    

[backbone.conv1, backbone, ]
[backbone.bn1]


In [20]:
dummy_input = torch.rand(5)
dummy_input.node = LinkedNode('input', dummy_input.shape)

In [21]:
getattr(dummy_input, 'node', 'default value')

LinkedNode

In [14]:
dummy_input.shape

torch.Size([5])

In [11]:
for n, m in trainer.manager.model.named_modules():
    print(n)


backbone
backbone.conv1
backbone.bn1
backbone.layer1
backbone.layer1.0
backbone.layer1.0.conv1
backbone.layer1.0.bn1
backbone.layer1.0.conv2
backbone.layer1.0.bn2
backbone.layer1.0.shortcut
backbone.layer1.1
backbone.layer1.1.conv1
backbone.layer1.1.bn1
backbone.layer1.1.conv2
backbone.layer1.1.bn2
backbone.layer1.1.shortcut
backbone.layer2
backbone.layer2.0
backbone.layer2.0.conv1
backbone.layer2.0.bn1
backbone.layer2.0.conv2
backbone.layer2.0.bn2
backbone.layer2.0.shortcut
backbone.layer2.0.shortcut.0
backbone.layer2.0.shortcut.1
backbone.layer2.1
backbone.layer2.1.conv1
backbone.layer2.1.bn1
backbone.layer2.1.conv2
backbone.layer2.1.bn2
backbone.layer2.1.shortcut
backbone.layer3
backbone.layer3.0
backbone.layer3.0.conv1
backbone.layer3.0.bn1
backbone.layer3.0.conv2
backbone.layer3.0.bn2
backbone.layer3.0.shortcut
backbone.layer3.0.shortcut.0
backbone.layer3.0.shortcut.1
backbone.layer3.1
backbone.layer3.1.conv1
backbone.layer3.1.bn1
backbone.layer3.1.conv2
backbone.layer3.1.bn2
bac

In [10]:
m

Model(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2